In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel


In [ ]:

adapter_path = []
# 경로 설정
base_model_path = "/home/infidea/rebirth-hjun/meeting_model_weights/kanana-1.5-8b-instruct-2505"
# adapter_path.append("/home/infidea/rebirth-hjun/KRAG_2025/trained_model/kanana-1.5-8b-instruct-2505-lora-20250715-1532")
adapter_path.append("/home/infidea/rebirth-hjun/KRAG_2025/trained_model/kanana-1.5-8b-instruct-2505-lora-20250728-1009")
# adapter_path.append("/home/infidea/rebirth-hjun/KRAG_2025/trained_model/kanana-1.5-8b-instruct-2505-lora-20250719-1945")
# merged_model_path = adapter_path.split("/")[-1]
# merged_model_path = "./output_dir"

# 디바이스 설정
device_arg = {"device_map": "auto"}

In [3]:
# 베이스 모델 로드
print(f"Loading base model from: {base_model_path}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)


Loading base model from: /home/infidea/rebirth-hjun/meeting_model_weights/kanana-1.5-8b-instruct-2505


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# LoRA 어댑터 로드 및 병합
print(f"Loading and merging PEFT from: {adapter_path}")
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

for i in adapter_path:
    merged_model_path = i.split("/")[-1]
    peft_model = PeftModel.from_pretrained(base_model, i)
    merged_model = peft_model.merge_and_unload()
    print(f"Saving merged model to: {merged_model_path}")
    merged_model .save_pretrained(merged_model_path)
    tokenizer.save_pretrained(merged_model_path)
    print("✅ 모델과 토크나이저 저장 완료")

Loading and merging PEFT from: ['/home/infidea/rebirth-hjun/KRAG_2025/trained_model/kanana-1.5-8b-instruct-2505-lora-20250728-1009']
Saving merged model to: kanana-1.5-8b-instruct-2505-lora-20250728-1009
✅ 모델과 토크나이저 저장 완료


In [9]:
# 토크나이저 로드 776542490961



In [3]:
# 변수 세팅
OUTPUT_PATH = '0728_4' + '-merge'
LORA_MERGE_CACHE = "/tmp"  # 로라 병합 캐시 저장 경로
CONFIG_YML = "/home/infidea/rebirth-hjun/KRAG_2025/modle_merge/config.yaml"  # config(설정) yml 파일 경로 
COPY_TOKENIZER = True  # 토크나이저 있는거 쓰기
LAZY_UNPICKLE = True  # 실험적인 저용량 모델 로더 기능 활성화
LOW_CPU_MEMORY = True  # 서버 성능 좋으면 True 하면 됨

In [4]:
# 패키지 임포트
import torch
import yaml

from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

# yml 파일 설정값 불러오기
with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options=MergeOptions(
        lora_merge_cache=LORA_MERGE_CACHE,
        cuda=torch.cuda.is_available(), # GPU 세팅
        copy_tokenizer=COPY_TOKENIZER, # 토크나이저
        lazy_unpickle=LAZY_UNPICKLE, 
        low_cpu_memory=LOW_CPU_MEMORY,
    ),
)
print("Done!")

Executing graph: 100%|██████████| 1457/1457 [04:43<00:00,  5.13it/s]

Done!
